In [1]:
# Librerias para la Manipulación de datos y visualización
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns


#LIbreria para pre-procesar datos (antes de entrenar el modelo)
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Librerias de modelos
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

#Librerias para evaluar modelos
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Importar el modelo de Clasificación
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
file_path = '/content/drive/MyDrive/SAP_Descuadres_Input/Consolidado_Maestro2.gsheet'

In [3]:
import gspread
from google.colab import auth
import pandas as pd
import google.auth

# Autenticar con Google
auth.authenticate_user()

# Obtener las credenciales autenticadas
creds, project = google.auth.default()

# Inicializar el cliente de gspread con las credenciales
gc = gspread.authorize(creds)

spreadsheet_name = 'Consolidado_Maestro2'
worksheet = gc.open(spreadsheet_name).sheet1

# Obtener todos los datos como una lista de listas
data = worksheet.get_all_values()

# Convertir los datos a un DataFrame de pandas
df = pd.DataFrame(data[1:], columns=data[0])

# Mostrar las primeras filas para verificar que se cargó correctamente
print(f"Datos cargados desde '{spreadsheet_name}'. Primeras 5 filas:")
print(df.head())

Datos cargados desde 'Consolidado_Maestro2'. Primeras 5 filas:
  Fecha Contable Centro Producto Copec   Material Cantidad  \
0     2024-12-31   C003          CUPON  HISTORICO        0   
1     2024-12-31   C003          MUEVO  HISTORICO   -0,005   
2     2024-12-31   C003        TCT_TAE  HISTORICO        0   
3     2024-12-31   C004          MUEVO  HISTORICO    -0,01   
4     2024-12-31   C005          CUPON  HISTORICO        0   

         Fecha Versión  
0  2025-12-15 13:19:17  
1  2025-12-15 13:19:17  
2  2025-12-15 13:19:17  
3  2025-12-15 13:19:17  
4  2025-12-15 13:19:17  


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61136 entries, 0 to 61135
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Fecha Contable  61136 non-null  datetime64[ns]
 1   Centro          61136 non-null  object        
 2   Producto Copec  61136 non-null  object        
 3   Material        61136 non-null  object        
 4   Cantidad        61136 non-null  float64       
 5   Fecha Versión   61136 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 2.8+ MB


In [18]:
# --- 1. CONFIGURACIÓN, LIMPIEZA DE DATOS Y CORRECCIÓN DE TIPOS ---

TARGET_COLUMN = 'Y_ALERTA_RIESGO'
UMBRAL = 0.05

# Convertir las columnas de fecha a datetime, especificando el formato día/mes/año
df['Fecha Contable'] = pd.to_datetime(df['Fecha Contable'], format='%Y-%m-%d')
df['Fecha Versión'] = pd.to_datetime(df['Fecha Versión'])

try:
    df['Cantidad'] = df['Cantidad'].astype(str).str.replace(',', '.', regex=False)
    df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors='coerce')
except Exception:
    df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors='coerce')

# Rellenar cualquier valor nulo introducido por la conversión forzada con 0.0
df['Cantidad'] = df['Cantidad'].fillna(0.0)

# Convertir 'Material' a string para OHE
df['Material'] = df['Material'].astype(str)


--- Data types ajustados y columna 'Cantidad' corregida. ---


In [25]:
# ====================================================================
# --- 2. VARIABLE OBJETIVO (Y) ---
# ====================================================================

# 1. Crear la columna de período (Columna auxiliar temporal)
df['Periodo'] = df['Fecha Contable'].dt.to_period('M')

# 2. Calcular la suma total de 'Cantidad' por 'Periodo' y 'Centro'
df_sum_by_center_month = df.groupby(['Periodo', 'Centro'])['Cantidad'].sum().reset_index()
df_sum_by_center_month.rename(columns={'Cantidad': 'Suma_Cantidad_Centro_Mes'}, inplace=True)

# 3. Aplicar la lógica de descuadre: |Suma_Cantidad| > UMBRAL
df_sum_by_center_month[TARGET_COLUMN] = (
    (df_sum_by_center_month['Suma_Cantidad_Centro_Mes'] < -UMBRAL) |
    (df_sum_by_center_month['Suma_Cantidad_Centro_Mes'] > UMBRAL)
).astype(int)

# 4. Fusionar el resultado del descuadre de vuelta al dataframe original
df = pd.merge(df, df_sum_by_center_month[['Periodo', 'Centro', TARGET_COLUMN]],
             on=['Periodo', 'Centro'],
             how='left')

# Asignar la variable objetivo 'y'
y = df[TARGET_COLUMN]

print(f"Número total de filas de Riesgo (1) en Y: {df[TARGET_COLUMN].sum()}")

--- Creando Variable Objetivo (Y) ---
Número total de filas de Riesgo (1) en Y: 19419


In [26]:
# ====================================================================
# --- 3. VARIABLES PREDICTORAS (X) ---
# ====================================================================

### A. Feature Engineering (Variables de Tiempo)
df['Dia_Semana'] = df['Fecha Contable'].dt.dayofweek
df['Dia_Mes'] = df['Fecha Contable'].dt.day

### B. One-Hot Encoding
features_to_encode = ['Centro', 'Producto Copec', 'Material']
numerical_features = ['Cantidad', 'Dia_Semana', 'Dia_Mes']

df_encoded = pd.get_dummies(df, columns=features_to_encode, drop_first=True)

### C. Definición de X
columns_to_drop = [
    TARGET_COLUMN,
    'Fecha Contable',
    'Fecha Versión',
    'Periodo',
    'Y_ALERTA_RIESGO_x',
    'Y_ALERTA_RIESGO_y'
]

# Usamos errors='ignore' para que no falle si estas columnas no existen.
X = df_encoded.drop(columns=columns_to_drop, errors='ignore')

print(f"Dimensiones de X: {X.shape}")


--- 3. Ingeniería de Variables (X) CORREGIDA ---
Dimensiones de X: (61136, 722)


In [27]:
# ====================================================================
# --- 4. ESCALADO Y DIVISIÓN DEL DATASET ---
# ====================================================================

TEST_SIZE = 0.2
RANDOM_STATE = 42

# 1. División del dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

# 2. Escalado de Variables Numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Tamaño de X_train (Escalado): {X_train_scaled.shape}")


--- 4. División y Escalado ---
Tamaño de X_train (Escalado): (48908, 722)


In [28]:
# ====================================================================
# --- 4.5 BALANCEO DE CLASES CON SMOTE ---
# ====================================================================
from imblearn.over_sampling import SMOTE

# Instanciar SMOTE
smote = SMOTE(random_state=42)

# Aplicar SMOTE SOLO a los datos de entrenamiento escalados
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"Tamaño de y_train original (0/1): {y_train.value_counts().to_dict()}")
print(f"Tamaño de y_train_smote balanceado (0/1): {y_train_smote.value_counts().to_dict()}")



Tamaño de y_train original (0/1): {0: 33373, 1: 15535}
Tamaño de y_train_smote balanceado (0/1): {0: 33373, 1: 33373}


In [29]:
# ====================================================================
# --- 5. ENTRENAMIENTO Y ANÁLISIS DEL MODELO  ---
# ====================================================================

# Crear y entrenar el modelo GBC
gbc_model_smote = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
# Entrenar con los datos balanceados
gbc_model_smote.fit(X_train_smote, y_train_smote)

# Predecir probabilidades y clases en el conjunto de PRUEBA (no balanceado)
y_pred_proba_smote = gbc_model_smote.predict_proba(X_test_scaled)[:, 1]
y_pred_smote = gbc_model_smote.predict(X_test_scaled)


### A. Métrica de Desempeño
print(f'ROC-AUC Score con SMOTE: {roc_auc_score(y_test, y_pred_proba_smote):.4f}')

### B. Matriz de Confusión
cm_smote = confusion_matrix(y_test, y_pred_smote)
print("\nMatriz de Confusión (Predicción vs. Real) con SMOTE:")
print(pd.DataFrame(cm_smote, index=['Real 0 (Normal)', 'Real 1 (Descuadre)'],
                   columns=['Pred 0 (Normal)', 'Pred 1 (Descuadre)']))

# Re-calcular Tasa de Falsos Negativos (la métrica que queremos reducir)
FN = cm_smote[1, 0]
TP = cm_smote[1, 1]
Tasa_FN = FN / (FN + TP) if (FN + TP) > 0 else 0
print(f"Tasa de Falsos Negativos (Riesgos Perdidos) con SMOTE: {Tasa_FN:.4f}")

### C. Importancia de las Variables
feature_importance_smote = pd.Series(gbc_model_smote.feature_importances_, index=X_train.columns)
feature_importance_smote.sort_values(ascending=False, inplace=True)
print('\nImportancia de las 10 Variables Principales (SMOTE):')
print(feature_importance_smote.head(10).to_string())

ROC-AUC Score con SMOTE: 0.7938

Matriz de Confusión (Predicción vs. Real) con SMOTE:
                    Pred 0 (Normal)  Pred 1 (Descuadre)
Real 0 (Normal)                6239                2105
Real 1 (Descuadre)             1234                2650
Tasa de Falsos Negativos (Riesgos Perdidos) con SMOTE: 0.3177

Importancia de las 10 Variables Principales (SMOTE):
Dia_Mes                 0.400976
Dia_Semana              0.154384
Cantidad                0.109804
Producto Copec_MUEVO    0.058947
Centro_N282             0.020326
Material_HISTORICO      0.016433
Centro_N197             0.012868
Centro_S089             0.012416
Centro_C601             0.011389
Centro_S615             0.011017


In [30]:
# ====================================================================
# --- 6. GENERACIÓN DE PREDICCIONES Y RANKING DE RIESGO ---
# ====================================================================


df_test_original = df.loc[X_test.index, ['Centro', 'Periodo']]

df_test_original['Probabilidad_Descuadre'] = y_pred_proba_smote
df_test_original['Alerta_Real'] = y_test.values


ranking_riesgo = df_test_original.groupby(['Periodo', 'Centro']).agg(
    Probabilidad_Maxima_Descuadre=('Probabilidad_Descuadre', 'max'),
    Alerta_Real=('Alerta_Real', 'max'),
    Numero_Transacciones=('Centro', 'size')
).reset_index()


ranking_riesgo_filtrado = ranking_riesgo[
    ranking_riesgo['Probabilidad_Maxima_Descuadre'] > 0.5
]


ranking_riesgo_filtrado = ranking_riesgo_filtrado.sort_values(
    by='Probabilidad_Maxima_Descuadre',
    ascending=False
)

print("\n--- 6. RANKING DE CENTROS CON MAYOR RIESGO DE DESCUADRE ---")
print("Predicción de riesgo basada en datos del conjunto de prueba (Probabilidad > 50%):")


ranking_riesgo_filtrado['Probabilidad_Maxima_Descuadre'] = (
    ranking_riesgo_filtrado['Probabilidad_Maxima_Descuadre'] * 100
).round(2).astype(str) + '%'

print(ranking_riesgo_filtrado.head(15).to_string())
print(f"\nTotal de Centros marcados como riesgo: {len(ranking_riesgo_filtrado)}")


--- 6. RANKING DE CENTROS CON MAYOR RIESGO DE DESCUADRE ---
Predicción de riesgo basada en datos del conjunto de prueba (Probabilidad > 50%):
      Periodo Centro Probabilidad_Maxima_Descuadre  Alerta_Real  Numero_Transacciones
1619  2025-05   N572                        89.14%            1                     1
1705  2025-05   S931                        88.22%            1                     1
2450  2025-07   S092                        88.11%            1                     1
2925  2025-08   G470                        87.77%            1                     3
2447  2025-07   S089                         87.1%            1                     1
546   2025-01   S615                        87.06%            1                     1
2987  2025-08   N197                        86.92%            1                     8
3071  2025-08   S188                        86.67%            1                     1
2336  2025-07   G470                        86.67%            1                    

In [31]:
# ====================================================================
# --- 7. PREDICCIÓN EN DATOS FUTUROS ---
# ====================================================================


df_future = df.copy()


df_future['Dia_Semana'] = df_future['Fecha Contable'].dt.dayofweek
df_future['Dia_Mes'] = df_future['Fecha Contable'].dt.day


features_to_encode = ['Centro', 'Producto Copec', 'Material']
numerical_features = ['Cantidad', 'Dia_Semana', 'Dia_Mes']

df_future_encoded = pd.get_dummies(df_future, columns=features_to_encode, drop_first=True)


X_train_cols = X_train.columns

X_future = df_future_encoded.reindex(columns=X_train_cols, fill_value=0)

print(f"Dimensiones de X para predicción: {X_future.shape}")


# 5. ESCALADO DE DATOS
X_future_scaled = scaler.transform(X_future)


# 6. PREDICCIÓN DE PROBABILIDADES
# modelo entrenado con SMOTE
future_probabilities = gbc_model_smote.predict_proba(X_future_scaled)[:, 1]

# 7. GENERACIÓN DEL RANKING DE RIESGO
df_future['Probabilidad_Descuadre'] = future_probabilities

# Crear el ranking agrupado por Periodo y Centro
ranking_futuro = df_future.groupby(['Periodo', 'Centro']).agg(
    Probabilidad_Maxima_Descuadre=('Probabilidad_Descuadre', 'max'),
    Numero_Transacciones=('Centro', 'size')
).reset_index()

# Filtrar y ordenar el ranking (usando un umbral de riesgo de 50%)
ranking_futuro_filtrado = ranking_futuro[
    ranking_futuro['Probabilidad_Maxima_Descuadre'] > 0.5
]

ranking_futuro_filtrado = ranking_futuro_filtrado.sort_values(
    by='Probabilidad_Maxima_Descuadre',
    ascending=False
)

# Formato de salida
ranking_futuro_filtrado['Probabilidad_Maxima_Descuadre'] = (
    ranking_futuro_filtrado['Probabilidad_Maxima_Descuadre'] * 100
).round(2).astype(str) + '%'

print("\n--- RANKING DE RIESGO PREDICTIVO (Futuro) ---")
print("Este es el informe de los Centros que *probablemente* descuadrarán:")
print(ranking_futuro_filtrado.head(15).to_string())
print(f"\nTotal de Centros predichos con riesgo (> 50%): {len(ranking_futuro_filtrado)}")



--- 7. Generando Predicciones para el Futuro ---
Dimensiones de X para predicción: (61136, 722)

--- RANKING DE RIESGO PREDICTIVO (Futuro) ---
Este es el informe de los Centros que *probablemente* descuadrarán:
      Periodo Centro Probabilidad_Maxima_Descuadre  Numero_Transacciones
4495  2025-06   S092                        89.17%                     3
3772  2025-05   N572                        89.14%                     3
5821  2025-08   N572                        89.14%                     9
5134  2025-07   N572                        88.95%                     9
4450  2025-06   N572                        88.87%                     3
5866  2025-08   S092                        88.63%                     7
3966  2025-05   S615                        88.43%                     4
4492  2025-06   S089                        88.42%                     3
6054  2025-08   S931                        88.22%                    24
4002  2025-05   S931                        88.22%       